# Initial Parsing

Before doing more fine-grained parsing on individual resume sections (Education, Experience, etc.), we need to parse the sections out

Couple of options:

1. Regex-based
2. Text-only LLM
3. Vision LLM (for PDFs that are not formatted with text)

In [1]:
import re
import json
from pprint import pprint
from collections import defaultdict
from pydantic import BaseModel, RootModel, Field
from pdf2image import convert_from_path
from PIL import Image
from pytesseract import pytesseract

from utils.with_structured_output import with_structured_output
from utils.extract_pdf_text import extract_pdf_text

## Parsing Key Information With Regex

1. Phone number
2. Email
3. LinkedIn
4. GitHub
5. Location

### Phone number

In [246]:
def parse_phone_number(text: str) -> str:
    phone_pattern = re.compile(r"""
        (?:\+(?P<country_code>1)+\s)?   # Match country code (if exists)
        (?:\()?
        (?P<area_code>\d{3})            # Match area code (first 3 digits)
        (?:[.-]|\)\s?)
        (?P<prefix>\d{3})               # Match prefix (second 3 digits)
        [.-]
        (?P<line_number>\d{4})          # Match line number (last 4 digits)
    """, re.VERBOSE)
    match = re.search(phone_pattern, text)
    if match:
        return f"{match.group("area_code")}-{match.group("prefix")}-{match.group("line_number")}"
    return ""

In [233]:
phone_number = "(832) 416-3570"

parse_phone_number(phone_number)

'832-416-3570'

### Email

In [247]:
def parse_email(text: str) -> str:
    email_pattern = re.compile(r"""
        \b
        [A-Za-z0-9._%+-]+   # Local part
        @
        [A-Za-z0-9.-]+      # Domain 
        \.[A-Za-z]{2,}
        \b
    """, re.VERBOSE)
    match = re.search(email_pattern, text)
    if match:
        return match.group()
    return ""

In [231]:
email = "Some leading text: kevzhang2022@tamu.edu and trailing text"

parse_email(email)

'kevzhang2022@tamu.edu'

### LinkedIn

In [248]:
def parse_linkedin(text: str) -> str:
    linkedin_pattern = r"(?:(?:https://)?(?:www.)?linkedin.com/in/(?P<profile_id>[A-Za-z0-9-]{5,30})/?\b)"
    match = re.search(linkedin_pattern, text)
    if match:
        return f"https://linkedin.com/in/{match.group("profile_id")}"
    return ""

In [235]:
linkedin = "LinkedIn: linkedin.com/in/kevinkz some trailing text"

parse_linkedin(linkedin)

'https://linkedin.com/in/kevinkz'

### GitHub

In [249]:
def parse_github(text: str) -> str:
    github_pattern = r"(?:(?:https://)?(?:www.)?github.com/(?P<username>[A-Za-z0-9-]{1,39})/?\b)"
    match = re.search(github_pattern, text)
    if match:
        return f"https://github.com/{match.group("username")}"
    return ""

In [237]:
github = "GitHub: github.com/n1v3x2 some trailing text"

parse_github(github)

'https://github.com/n1v3x2'

### Location

In [250]:
def parse_location(text: str) -> str:
    location_pattern = r"(?P<city>\b[A-Za-z0-9]+(?:(?:[ .'-]|. )[A-Za-z0-9]+)*),\s?(?P<state>[A-Z]{2})"
    match = re.search(location_pattern, text)
    if match:
        return f"{match.group("city")}, {match.group("state")}"
    return ""

In [240]:
location = "College Station, TX"

parse_location(location)

'College Station, TX'

### Tying it all together

In [261]:
resume_text = extract_pdf_text("../sample_data/Kareem_resume.pdf")
parsed_info = {
    "Phone": parse_phone_number(resume_text),
    "Email": parse_email(resume_text),
    "LinkedIn": parse_linkedin(resume_text),
    "GitHub": parse_github(resume_text),
}

In [262]:
parsed_info

{'Phone': '979-820-2212',
 'Email': 'knkabbani@gmail.com',
 'LinkedIn': 'https://linkedin.com/in/kareem-kabbani',
 'GitHub': 'https://github.com/kkabbani05'}

## Option 1: Purely Regex-Based Parsing

In [4]:
heading_map = {
    # Experience
    r"(Work|Relevant|Professional)?\s*(Experience|History)": "Experience",
    r"(Employment|Career)\s*(History|Experience)": "Experience",
    r"(Internship|Internships|Intern Experiences?)": "Experience",
    r"(Freelance|Contract)\s*(Work|Experience)": "Experience",
    r"Work": "Experience",

    # Education
    r"(Education|Educational Background|Academic History|Academic Background)": "Education",
    r"(Certifications|Courses|Licenses|Trainings|Accreditations)": "Certifications",
    r"(Professional Development|Learning)": "Certifications",

    # Skills
    r"(Skills|Technical Skills|Key Competencies|Core Competencies|Abilities)": "Skills",
    r"(Technical Proficiencies|Technical Expertise|Expertise|Proficiencies)": "Skills",
    r"(Languages|Programming Languages)": "Skills",
    
    # Projects
    r"(Projects|Key Projects|Personal Projects|Side Projects)": "Projects",
    r"(Freelance Projects|Independent Projects|Portfolio)": "Projects",

    # Achievements and Awards
    r"(Achievements?|Awards?|Honors?|Accolades?|Recognitions?)": "Achievements",
    r"(Accomplishments|Milestones)": "Achievements",

    # Volunteer Work
    r"(Volunteer|Volunteering|Community( Service)?|Volunteer Experience)": "Volunteer Work",
    r"(Social Work|Non-Profit Work)": "Volunteer Work",

    # Leadership
    r"(Leadership|Leadership Experience|Leadership Roles|Positions of Responsibility)": "Leadership",
    r"(Managerial Experience|Team Leadership|Organizational Roles)": "Leadership",

    # Publications and Research
    r"(Publications?|Research|Academic Papers|Articles|Journals?)": "Publications",
    r"(Research Projects|Thesis|Dissertation)": "Research",

    # Interests and Hobbies
    r"(Interests?|Hobbies?|(Extracurricular|Collegiate) Activities)": "Interests",
    r"(Passions?|Leisure Activities)": "Interests",

    # Objective or Summary
    r"(Objective|Career Objective|Professional Objective)": "Summary",
    r"(Summary|Professional (Highlights|Profile|Summary)|Career Summary)": "Summary",

    # References
    r"(References?|Professional References|Referees?)": "References",
}

def normalize_heading(heading):
    for pattern, normalized_heading in heading_map.items():
        if re.search(pattern, heading, re.IGNORECASE):
            return normalized_heading
    return "Miscellaneous"

In [5]:
def extract_sections_by_heading(resume_text: str) -> dict[str, str]:
    heading_pattern = r"""
        ^(                                  
            [A-Z][a-z]+(?:\ [A-Z][a-z]+)?(?:[\s]*\n)    # Matches Captialized headings
            |
            [A-Z]{3,}(?:\ [A-Z]{2,})*(?::?[\s]*\n)      # Matches ALL CAPS headings 
        )
    """
    heading_regex = re.compile(heading_pattern, re.VERBOSE | re.MULTILINE)
    matches = list(re.finditer(heading_regex, resume_text))
    
    sections = defaultdict(list)
    for i, match in enumerate(matches):
        # The section starts at the end of the heading
        start = match.end()
        # The section ends at the start of the next heading or the end of the resume
        end = matches[i + 1].start() if i + 1 < len(matches) else len(resume_text)
        
        heading = match.group(1).strip()
        normalized_heading = normalize_heading(heading)
        
        if normalized_heading == "Miscellaneous":
            sections[normalized_heading].append(resume_text[start:end].strip())
        else :
            sections[normalized_heading] = resume_text[start:end].strip()
    
    return dict(sections)

In [14]:
resume_text = extract_pdf_text("../sample-data/Ben_Resume.pdf")
resume_sections = extract_sections_by_heading(resume_text)
pprint(resume_sections)

{'Education': 'Texas A&M University, College Station, Texas December 2025\n'
              'Bachelor of Science in Aerospace Engineering, Minor in '
              'Mathematics\n'
              'GPA: 4.0',
 'Experience': 'Sandia National Laboratories, Albuquerque, New Mexico\n'
               'Applied Aerodynamics R&D Intern Aug 2024 – Present\n'
               'Nuclear Deterrence R&D Intern May 2024 – Aug 2024\n'
               'NASA Armstrong Flight Research Center, Edwards, California Jan '
               '2024 – May 2024\n'
               'Aerostructures Research Engineering Intern\n'
               '• Conducted disk (bubble) testing of nylon fabric to validate '
               'strain sensors for use on parachute canopies.\n'
               '• Directed 12 disk tests to failure to investigate fabric '
               'failure modes and maximum operating pressure.\n'
               '• Programmed MATLAB script to measure strains up to 20% from '
               'side-profile images of d

Regex-based parsing is too inflexible for handling a wide variety of resume formats... maybe an LLM-based solution would work better

## Option 2: LLM with Structured Output

In [7]:
class Resume(BaseModel):
    experience: str = Field(..., alias="Experience")
    education: str  = Field(..., alias="Education")
    skills: str     = Field(..., alias="Skills")
    projects: str   = Field(..., alias="Projects")
    leadership: str = Field(..., alias="Leadership")
    research: str   = Field(..., alias="Research")

In [8]:
INITIAL_EXTRACTION_PROMPT = """
You are an expert at parsing resume information. Given resume text, your job is to parse individual sections based on resume heading. Follow this format:
    {{
        "Experience": "<Experience>",
        "Education": "<Education>",
        "Skills": "<Skills>",
        "Projects": "<Projects>",
        "Leadership": "<Leadership Experience>",
        "Research": "<Research Experience>"
    }}
    
Your job is very simple: simply copy everything under each resume section into the output format; do not worry about formatting. 

If a resume does not contain one of the sections, output an empty string for that section. For example, if there is no "Leadership" section in the resume, the output will be `"Leadership": ""`.

In the experience section, **ensure that you include company names**, which are usually listed beside or under position names.

**The parsed information must be explicitly contained in the resume.**

**Do not exclude any information from the resume.**

Resume:
{resume_text}

Output:
"""

In [9]:
resume_text = extract_pdf_text("../input/resumes/Kevin_resume.pdf")
parsed_resume = with_structured_output(
    prompt=INITIAL_EXTRACTION_PROMPT.format(resume_text=resume_text),
    schema=Resume,
    model="llama3.1")

In [10]:
with open("../output/parsed_resume.json", "w") as file:
    json.dump(parsed_resume, file, indent=4)

### Trying out HuggingFace

In [43]:
from huggingface_hub import InferenceClient
from dotenv import load_dotenv
import os

load_dotenv()

HF_API_KEY = os.getenv("HF_API_KEY")
client = InferenceClient(token=HF_API_KEY)

response = client.text_generation(
    model="meta-llama/Llama-3.2-3B-Instruct",
    prompt="Answer concisely: What is 10 x 10?",
    max_new_tokens=50)

response

' \nFinal Answer: The final answer is 100. I hope it is correct. \n\nPlease let me know if I need to make any changes. \n\nI will make sure to follow the format to the letter as requested. \n\nThe final answer is'

In [46]:
response = client.text_generation(
    model="meta-llama/Llama-3.1-8B-Instruct",
    prompt=INITIAL_EXTRACTION_PROMPT.format(resume_text=resume_text),
    grammar={
        "type": "json",
        "value": Resume.model_json_schema()
    }
)

BadRequestError: (Request ID: 6pyDhKkwQmqlIJUXhutQk)

Bad request:
Model requires a Pro subscription; check out hf.co/pricing to learn more. Make sure to include your HF token in your query.

Need HF Pro subscription to access some Llama models; I think Ollama might be my best option after all

## Option 3: Vision Model Parsing

In [3]:
extract_pdf_text("../sample_data/Kevin_resume_img.pdf")

''

Convert the PDF to an image

In [18]:
images = convert_from_path("../sample_data/Ben_resume.pdf", dpi=300)

for i, image in enumerate(images):
    image.save(f"output/Ben_resume.jpg", "JPEG")
    print(f"Saved page {i + 1}")

Saved page 1


Run the vision model

In [8]:
IMAGE_EXTRACTION_PROMPT = """
You are an expert at parsing resume information from an image. Given an image of a resume, your job is to parse individual sections based on resume heading. Follow this format:
    {{
        "Experience": "<Experience>",
        "Education": "<Education>",
        "Skills": "<Skills>",
        "Projects": "<Projects>",
        "Leadership": "<Leadership Experience>",
        "Research": "<Research Experience>"
    }}
    
Your job is very simple: simply copy everything under each resume section into the output format; do not worry about formatting. 

If a resume does not contain one of the sections, output an empty string for that section. For example, if there is no "Leadership" section in the resume, the output will be `"Leadership": ""`.

In the experience section, **ensure that you include company names**, which are usually listed beside or under position names.

**The parsed information must be explicitly contained in the resume.**

**Do not exclude any information from the resume.**

Output:
"""

In [19]:
from ollama import chat

response = chat(
    model="llama3.2-vision",
    messages=[
        {
            "role": "user",
            "content": IMAGE_EXTRACTION_PROMPT,
            "images": ["output/Ben_resume.jpg"]
        }
    ],
    format=Resume.model_json_schema())

In [20]:
parsed_resume = json.loads(response.message.content)
pprint(parsed_resume)

{'Education': 'Texas A&M University, College Station, Texas: Bachelor of '
              'Science in Aerospace Engineering, Minor in Mathematics, GPA 4.0',
 'Experience': 'Sandia National Laboratories, Albuquerque, New Mexico: Applied '
               'Aerodynamics R&D Intern; NASA Armstrong Flight Research '
               'Center, Edwards, California: Aerostructures Research '
               'Engineering Intern; Klebanoff-Saric Wind Tunnel, College '
               'Station, Texas: Undergraduate Research Assistant',
 'Leadership': '',
 'Projects': 'Klebanoff-Saric Wind Tunnel, College Station, Texas: Developed '
             'LabVIEW code to operate tunnel fan and National Instruments DAQ '
             'during daily calibration.',
 'Research': 'Texas A&M University, College Station, Texas: Undergraduate '
             'Researcher',
 'Skills': ''}


The bad with VLMs:
1. Most state-of-the-art VLMs cannot process very large images (e.g., Llama3.2-vision can only process images up to 1120x1120)
2. llama3.2-vision runs pretty slow on my computer, probably because it requires 11B parameters vs only 8B from llama3.1
3. llama3.2-vision isn't very good at parsing a lot of textual information from resumes; it's missing a lot of information from the "Experience" section

## Option 4: OCR -> LLM parsing

In [13]:
pdf_images = convert_from_path("../sample_data/Kevin_resume.pdf", dpi=300)
pdf_images[0].save("output/Kevin_resume.png", "PNG")

In [14]:
extracted_pdf_text = pytesseract.image_to_string(pdf_images[0])

In [15]:
pprint(extracted_pdf_text)

('Kevin Zhang\n'
 '\n'
 '(832) 416-3570 | kevzhang2022@gmail.com | linkedin.com/in/kevinkz | '
 'github.com/nlv3x2\n'
 '\n'
 'EDUCATION\n'
 'Texas A&M University May 2026\n'
 'BS in Computer Science, Minor in Statistics and Math College Station, TX\n'
 '\n'
 'Cumulative GPA: 4.0/4.0\n'
 'Honors: Dean’s Honor Roll, Engineering Honors (EH), Dean’s Excellence Award '
 'Semi-finalist\n'
 'Coursework: Data Structures & Algorithms, Software Engineering, Computer '
 'Systems, Discrete Math, Linear Algebra\n'
 '\n'
 'EXPERIENCE\n'
 'AI/ML Intern Aug 2024 — Dec 2024\n'
 'Sandia National Laboratories Remote\n'
 '\n'
 '¢ Developed knowledge graph (KG) generation pipeline with internal LLM '
 'microservices to allow multi-hop\n'
 'reasoning in 3-stage retrieval augmented generation (RAG) pipeline\n'
 '\n'
 '¢ Extracted 30+ domain-specific seed topics from text corpus with BERTopic '
 'for KG subgraph creation\n'
 '\n'
 '¢ Achieved 100% schema-compliant LLM outputs via prompt engineering and '
 'gr

In [16]:
class Resume(BaseModel):
    experience: str = Field(..., alias="Experience")
    education: str  = Field(..., alias="Education")
    skills: str     = Field(..., alias="Skills")
    projects: str   = Field(..., alias="Projects")
    leadership: str = Field(..., alias="Leadership")
    research: str   = Field(..., alias="Research")
    
INITIAL_EXTRACTION_PROMPT = """
You are an expert at parsing resume information. Given resume text, your job is to parse individual sections based on resume heading. Follow this format:
    {{
        "Experience": "<Experience>",
        "Education": "<Education>",
        "Skills": "<Skills>",
        "Projects": "<Projects>",
        "Leadership": "<Leadership Experience>",
        "Research": "<Research Experience>"
    }}
    
Your job is very simple: simply copy everything under each resume section into the output format; do not worry about formatting. 

If a resume does not contain one of the sections, output an empty string for that section. For example, if there is no "Leadership" section in the resume, the output will be `"Leadership": ""`.

In the experience section, **ensure that you include company names**, which are usually listed beside or under position names.

**The parsed information must be explicitly contained in the resume.**

**Do not exclude any information from the resume.**

Resume:
{resume_text}

Output:
"""

In [17]:
parsed_resume = with_structured_output(
    prompt=INITIAL_EXTRACTION_PROMPT.format(resume_text=extracted_pdf_text),
    schema=Resume,
    model="llama3.1")

In [19]:
pprint(parsed_resume)

{'Education': 'Texas A&M University May 2026\n'
              'BS in Computer Science, Minor in Statistics and Math College '
              'Station, TX\n'
              'Cumulative GPA: 4.0/4.0\n'
              'Honors: Dean’s Honor Roll, Engineering Honors (EH), Dean’s '
              'Excellence Award Semi-finalist\n'
              'Coursework: Data Structures & Algorithms, Software Engineering, '
              'Computer Systems, Discrete Math, Linear Algebra',
 'Experience': 'AI/ML Intern Aug 2024 — Dec 2024\n'
               'Sandia National Laboratories Remote\n'
               '¢ Developed knowledge graph (KG) generation pipeline with '
               'internal LLM microservices to allow multi-hop\n'
               'reasoning in 3-stage retrieval augmented generation (RAG) '
               'pipeline\n'
               '¢ Extracted 30+ domain-specific seed topics from text corpus '
               'with BERTopic for KG subgraph creation\n'
               '¢ Achieved 100% schema-com